# Quick Start

Below is a simple demo of interaction with the environment of the vm sheduling sceanrio.

In [1]:
from maro.simulator import Env
from maro.simulator.scenarios.vm_scheduling import AllocateAction, DecisionPayload

env = Env(scenario="vm_scheduling", topology="azure.2019.10k", start_tick=0, durations=8638, snapshot_resolution=1)

metrics: object = None
decision_payload: DecisionPayload = None
is_done: bool = False

while not is_done:
    action: AllocateAction = None
    metrics, decision_payload, is_done = env.step(action)

print(metrics)

{'total_vm_requests': 10000, 'total_energy_consumption': 863800.0, 'successful_allocation': 0, 'successful_completion': 0, 'failed_allocation': 0, 'total_latency': Latency(Agent=0, Resource=0), 'total_oversubscriptions': 0}


# Environment of the vm scheduling

To initialize an environment, you need to specify the values of several parameters:

- **scenario**: The target scenario of this Env.
  - `vm_scheduling` denotes for the virtual machine scheuling.
- **topology**: The target topology of this Env. As shown below, you can get the predefined topology list by calling `get_topologies(scenario='vm_scheduling')`
- **start_tick**: The start tick of this Env. In vm_scheduling scenario, 1 tick respresents as 5 minutes in real-time.
  - In the demo above, `start_tick=0` indicates a simulation start from the beginning of the given topology.
- **durations**: The duration of thie Env, in the unit of tick/5 minutes.
  - In the demo above, `durations=8638` indicates a simulation length of roughly 30 days (30d * 24h * 60m / 5).
- **snapshot_resolution**: The time granularity of maintaining the snapshots of the environments, in the unit of tick/5 minutes.
  - In the demo above, `snapshot_resolution=1` indicates that a snapshot will be created and saved every tick during the simulation.
  
You can get all available scenarios and topologies by calling:

In [2]:
from maro.simulator.utils import get_scenarios, get_topologies
from pprint import pprint
from typing import List

scenarios: List[str] = get_scenarios()
topologies: List[str] = get_topologies(scenario='vm_scheduling')

pprint(f'The available scenarios in MARO:')
pprint(scenarios)

print()
pprint(f'The predefined topologies in VM Scheuling:')
pprint(topologies)

'The available scenarios in MARO:'
['cim', 'citi_bike', 'vm_scheduling']

'The predefined topologies in VM Scheuling:'
['azure.2019.10k', 'azure.2019.336k']


Once you created an instance of the environment, you can easily access the real-time information of this environment, like:

In [3]:
from maro.backends.frame import SnapshotList
from maro.simulator import Env
from pprint import pprint


# Initialize an Env for vm_scheduling scenario.
env = Env(scenario="vm_scheduling", topology="azure.2019.10k", start_tick=0, durations=8638, snapshot_resolution=1)

# The current tick.
tick: int = env.tick
print(f"The current tick: {tick}.")

# The current frame index, which indicates the index of current frame in the snapshot-list.
frame_index: int = env.frame_index
print(f"The current frame index: {frame_index}.")

# The whole snapshot-list of the environment, snapshots are taken in the granularity of the given snapshot_resolution.
# The example of how to use the snapshot will be shown later.
snapshot_list: SnapshotList = env.snapshot_list
print(f"There will be {len(snapshot_list)} snapshots in total.")

# The summary information of the environment.
summary: dict = env.summary
print(f"\nEnv Summary:")
pprint(summary)

# The metrics of the environment
metrics: dict = env.metrics
print(f"\nEnv Metrics:")
pprint(metrics)

The current tick: 0.
The current frame index: 0.
There will be 8638 snapshots in total.

Env Summary:
{'event_payload': {'PENDING_DECISION': ['valid_pms',
                                        'vm_id',
                                        'vm_cpu_cores_requirement',
                                        'vm_memory_requirement',
                                        'remaining_buffer_time'],
                   'REQUEST': ['vm_info', 'remaining_buffer_time']},
 'node_detail': {'pms': {'attributes': {'cpu_cores_allocated': {'slots': 1,
                                                                'type': 'i2'},
                                        'cpu_cores_capacity': {'slots': 1,
                                                               'type': 'i2'},
                                        'cpu_utilization': {'slots': 1,
                                                            'type': 'f'},
                                        'energy_consumption': {'slots': 1,

# Interaction with the environment

Before starting interaction with the environment, we need to know **DecisionPayload** and **Action** first.

## DecisionPayload

Once the environment need the agent's response to promote the simulation, it will throw an PendingDecision event with the **DecisionPayload**. In the scenario of vm_scheduling, the information of DecisionPayload is listed as below:
- **valid_pms** (List): The list of the PM ID that is considered as valid (Its CPU and memory resource is enough for the incoming VM).
- **vm_id** (int): The VM ID of the incoming VM (VM that is waiting for the placement).
- **vm_cpu_cores_requirement** (int): The CPU cores that is requested by the incoming VM.
- **vm_memory_requirement** (int): The memory resource that is reqeusted by the incoming VM.
- **remaining_buffer_time** (int): The remaining buffer time for the VM placement. The initial buffer time budget can be set in the `config.yml`.

## Action
Once get a **DecisionPayload** from the envirionment, the agent should respond with an Action. Valid Action includes None, AllocateAction, and PostponeAction.

- None: Do nothing.
- AllocateAction:
  - vm_id (int): The ID of the VM that is waiting for the allocation.
  - pm_id (int): The ID of the PM where the VM is scheduled to place at.
- PostponeAction:
  - vm_id (int): The ID of the VM that is waiting for the placement.
  - postpone_frequency (int): The number of times that the placement to be postponed. The unit is `DELAY_DURATION`. 1 means delay 1 `DELAY_DURATION`, which can be set in the `config.yml`.
 
## Generate random actions

The demo code in the Quick Start part has shown an interaction mode that doing nothing(responding with None action). Here we read the detailed information about the DecisionPayload and randomly generate choose a vailable PM.

In [5]:
import random

from maro.simulator import Env
from maro.simulator.scenarios.vm_scheduling import AllocateAction, DecisionPayload, PostponeAction

# Initialize an Env for vm_scheduling scenario
env = Env(scenario="vm_scheduling", topology="azure.2019.10k", start_tick=0, durations=8638, snapshot_resolution=1)

metrics: object = None
decision_event: DecisionPayload = None
is_done: bool = False
action: AllocateAction = None
    
# Start the env with a None Action
metrics, decision_event, is_done = env.step(None)

while not is_done:
    valid_pm_num: int = len(decision_event.valid_pms)
    if valid_pm_num <= 0:
        # No valid PM now, postpone.
        action: PostponeAction = PostponeAction(
            vm_id=decision_event.vm_id,
            postpone_frequency=1
        )
    else:
        # Randomly choose a vailable PM.
        random_idx = random.randint(0, valid_pm_num - 1)
        pm_id = decision_event.valid_pms[random_idx]
        action: AllocateAction = AllocateAction(
            vm_id=decision_event.vm_id,
            pm_id=pm_id
        )
    metrics, decision_event, is_done = env.step(action)

print(f"[Random] Topology: azure.2019.10k. Total ticks: 8638. Start tick: 0")
print(metrics)

[Random] Topology: azure.2019.10k. Total ticks: 8638. Start tick: 0
{'total_vm_requests': 10000, 'total_energy_consumption': 2426863.906832576, 'successful_allocation': 9877, 'successful_completion': 9058, 'failed_allocation': 123, 'total_latency': Latency(Agent=0, Resource=0), 'total_oversubscriptions': 0}


# Get the environment observation

You can also implement other strategies or build models to take action. At this time, real-time information and historical records of the environment are very important for making good decisions. In this case, the the environment snapshot list is exactly what you need.

The information in the snapshot list is indexed by 3 dimensions:

- A frame index (list). (int / List[int]) Empty indicates for all time slides till now.
- A PM id (list). (int / List[int]) Empty indicates for all PMs/agents.
- An Attribute name (list). (str / List[str]) You can get all available attributes in env.summary as shown before.

The return value from the snapshot list is a `numpy.ndarray` with shape (`num_frame` `num_pms` `num_attribute`, ).

More detailed introduction to the snapshot list is [here](https://maro.readthedocs.io/en/latest/key_components/data_model.html#advanced-features).

In [6]:
from pprint import pprint

from maro.simulator import Env

# Initialize an Env for vm_scheduling scenario
env = Env(scenario="vm_scheduling", topology="azure.2019.10k", start_tick=0, durations=8638, snapshot_resolution=1)

# To get the attribute list that can be accessed in snapshot_list
pprint(env.summary['node_detail'], depth=2)
print()
# The attribute list of stations
pprint(env.summary['node_detail']['pms'])

{'pms': {'attributes': {...}, 'number': 100}}

{'attributes': {'cpu_cores_allocated': {'slots': 1, 'type': 'i2'},
                'cpu_cores_capacity': {'slots': 1, 'type': 'i2'},
                'cpu_utilization': {'slots': 1, 'type': 'f'},
                'energy_consumption': {'slots': 1, 'type': 'f'},
                'id': {'slots': 1, 'type': 'i'},
                'memory_allocated': {'slots': 1, 'type': 'i2'},
                'memory_capacity': {'slots': 1, 'type': 'i2'}},
 'number': 100}


In [7]:
from pprint import pprint

from maro.simulator import Env
from maro.simulator.scenarios.vm_scheduling import AllocateAction, DecisionPayload, PostponeAction

env = Env(scenario="vm_scheduling", topology="azure.2019.10k", start_tick=0, durations=8638, snapshot_resolution=1)

metrics: object = None
decision_event: DecisionPayload = None
is_done: bool = False
action: AllocateAction = None
    
metrics, decision_event, is_done = env.step(None)

while not is_done:
    # This demo is used to show how to retrieve the information from the snapshot,
    # we terminate at 500th tick and see the output of the environment.
    if env.frame_index >= 500:
        valid_pm_info = env.snapshot_list["pms"][
            env.frame_index:decision_event.valid_pms:["cpu_cores_capacity", "cpu_cores_allocated"]
        ].reshape(-1, 2)
        # Show the first 10 machines. {cpu_cores_capacity: cpu_cores_allocated}
        pprint(valid_pm_info[:10])
        break
        
    valid_pm_num: int = len(decision_event.valid_pms)
    if valid_pm_num <= 0:
        # No valid PM now, postpone.
        action: PostponeAction = PostponeAction(
            vm_id=decision_event.vm_id,
            postpone_frequency=1
        )
    else:
        # Query the snapshot list of this environment to get the PM's capacity and allocated cores from snapshot.
        valid_pm_info = env.snapshot_list["pms"][
            env.frame_index:decision_event.valid_pms:["cpu_cores_capacity", "cpu_cores_allocated"]
        ].reshape(-1, 2)
        # Calculate to get the remaining cpu cores.
        cpu_cores_remaining = valid_pm_info[:, 0] - valid_pm_info[:, 1]
        # Choose the one with the closet remaining CPU.
        chosen_idx = 0
        minimum_remaining_cpu_cores = cpu_cores_remaining[0]
        for i, remaining in enumerate(cpu_cores_remaining):
            if remaining < minimum_remaining_cpu_cores:
                chosen_idx = i
                minimum_remaining_cpu_cores = remaining
        # Take action to place on the closet pm.
        action: AllocateAction = AllocateAction(
            vm_id=decision_event.vm_id,
            pm_id=decision_event.valid_pms[chosen_idx]
        )
    metrics, decision_event, is_done = env.step(action)

array([[32., 24.],
       [32., 26.],
       [32., 26.],
       [32., 20.],
       [32.,  4.],
       [32.,  4.],
       [32.,  0.],
       [32.,  0.],
       [32.,  0.],
       [32.,  0.]], dtype=float32)
